# Preprocess data

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import os
# Use CPU if no GPU
# os.environ['PSQI_DEVICE'] = 'cuda:1'
os.environ['PSQI_DEVICE'] = 'cpu'
os.environ['PSQI_RESULTS_DIR'] = os.path.join(os.getcwd(), 'results')
os.environ['PSQI_DATA_DIR'] = os.path.join(os.getcwd(), 'data')

In [ ]:
from psqi.preprocess import parse_raw_data, scale_data, _get_y_limits

df_all, df_X, df_Y, norms = parse_raw_data()
X, Y, X_scaler, Y_scaler, scaled_norms = scale_data(df_X, df_Y, norms)

# Learning the models

In [ ]:
params = [
    {'rank': 3, 'num_mixtures': 1, 'n_splits': 5, 'max_iter': 100},
    {'rank': 5, 'num_mixtures': 1, 'n_splits': 5, 'max_iter': 100},
    {'rank': 7, 'num_mixtures': 1, 'n_splits': 5, 'max_iter': 100},
    {'rank': 10, 'num_mixtures': 1, 'n_splits': 5, 'max_iter': 100},
    {'rank': 15, 'num_mixtures': 1, 'n_splits': 5, 'max_iter': 100},
    {'rank': 3, 'num_mixtures': 2, 'n_splits': 5, 'max_iter': 100},
    {'rank': 5, 'num_mixtures': 2, 'n_splits': 5, 'max_iter': 100},
    {'rank': 7, 'num_mixtures': 2, 'n_splits': 5, 'max_iter': 100},
    {'rank': 10, 'num_mixtures': 2, 'n_splits': 5, 'max_iter': 100},
    {'rank': 15, 'num_mixtures': 2, 'n_splits': 5, 'max_iter': 100},
    {'rank': 3, 'num_mixtures': 3, 'n_splits': 5, 'max_iter': 100},
    {'rank': 5, 'num_mixtures': 3, 'n_splits': 5, 'max_iter': 100},
    {'rank': 7, 'num_mixtures': 3, 'n_splits': 5, 'max_iter': 100},
    {'rank': 10, 'num_mixtures': 3, 'n_splits': 5, 'max_iter': 100},
    {'rank': 15, 'num_mixtures': 3, 'n_splits': 5, 'max_iter': 100},
    {'rank': 3, 'num_mixtures': 4, 'n_splits': 5, 'max_iter': 100},
    {'rank': 5, 'num_mixtures': 4, 'n_splits': 5, 'max_iter': 100},
    {'rank': 7, 'num_mixtures': 4, 'n_splits': 5, 'max_iter': 100},
    {'rank': 10, 'num_mixtures': 4, 'n_splits': 5, 'max_iter': 100},
    {'rank': 15, 'num_mixtures': 4, 'n_splits': 5, 'max_iter': 100},
    {'rank': 3, 'num_mixtures': 5, 'n_splits': 5, 'max_iter': 100},
    {'rank': 5, 'num_mixtures': 5, 'n_splits': 5, 'max_iter': 100},
    {'rank': 7, 'num_mixtures': 5, 'n_splits': 5, 'max_iter': 100},
    {'rank': 10, 'num_mixtures': 5, 'n_splits': 5, 'max_iter': 100},
    {'rank': 15, 'num_mixtures': 5, 'n_splits': 5, 'max_iter': 100},
]

In [ ]:
from ipypb import track
from psqi.experiment import run_experiment

for kwargs in track(params, total=len(params)):
    print(kwargs)
    run_experiment(X=X, Y=Y, X_scaler=X_scaler, Y_scaler=Y_scaler, **kwargs, random_state=0)

# Selecting the best model

In [ ]:
from psqi.experiment import read_results_and_models, get_results_R2_per_params

results, models = read_results_and_models(params)

In [ ]:
# R^2 score for each model averaged across every split and parameter
R2 = {
    key: np.array([
        entry['R2_orig'] 
        for entry in entries
    ])
    for key, entries in results.items()
}
results_R2_per_params = get_results_R2_per_params(R2=R2, params=params, y_columns=df_Y.columns)
results_R2_per_params

In [ ]:
# Drop all elements, which have maximum R2-score across all of the models less than 0
predicted_well = np.where(results_R2_per_params.iloc[:, 1:].max(axis=1) >= 0)[0]
predicted_well_cols = results_R2_per_params.iloc[:, 0][predicted_well]
print('Well-predicted params', list(predicted_well_cols))
R2_good = {
    k: v[:, predicted_well]
    for k,v in R2.items()
}

In [ ]:
import pandas as pd

# 1) For each model and each parameter compute average R^2 across all splits
# 2) For each model compute average R^2 across all well-predicted parameters
R2_per_models = np.array([v.mean(axis=0) for v in R2_good.values()]).mean(axis=1)
results_R2_per_models = pd.DataFrame()
results_R2_per_models['R2'] = R2_per_models.round(3)
ranks = [kwargs['rank'] for kwargs in params]
qs = [kwargs['num_mixtures'] for kwargs in params]
results_R2_per_models['r'] = ranks
results_R2_per_models['Q'] = qs
results_R2_per_models['#'] = results_R2_per_models.index.astype(int)
scores = results_R2_per_models['R2']

results_R2_per_models

In [ ]:
from psqi.experiment import get_key, get_dir
import dill as pickle

# Choose the best combination of parameters (rank and number of mixtures)
best_kwargs = params[np.argmax(scores)]
best_key = get_key(**best_kwargs, split=None)
print('Best parameters', best_kwargs)
best_R2 = np.array([
    result['R2']
    for result in results[best_key]
])
# Choose the best split for this model
best_split = np.argmax(np.array(best_R2).mean(axis=1))
best_dir = get_dir(**best_kwargs, split=best_split)
with open(best_dir + '/ix.pickle', 'rb') as fin:
    best_ix = pickle.load(fin)

# Get the best model
best_model = models[best_key][best_split]['model']
best_likelihood = models[best_key][best_split]['likelihood'] 

best_covar = best_model.covar_module.task_covar_module.covar_matrix.numpy()
best_mk = best_model.covar_module.data_covar_module

## Best model estimated noise

In [ ]:
# Noise is measured with interquartile range relative to the normative range

best_noise = best_likelihood.noise_covar.noise.detach().cpu().numpy()
best_noise = np.sqrt(best_noise) * 0.6745 * 2
best_noise = best_noise / np.diff(scaled_norms).flatten() * 100
inds = np.argsort(best_R2.mean(axis=0))
df_noise = pd.DataFrame(best_noise[inds], columns=['noise, %'], index=np.array(df_Y.columns)[inds])
df_noise

## Best model results

In [ ]:
best_results = pd.DataFrame()
best_results['R2-mean'] = best_R2.mean(axis=0).round(3)
best_results['R2-std'] = best_R2.std(axis=0).round(3)
best_results['Y-mean'] = Y.mean(axis=0)
best_results['Y-std'] = Y.std(axis=0)
best_results.index = df_Y.columns.tolist()
best_results = best_results.sort_values('R2-mean').iloc[::-1]
best_results

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,7))
t = np.arange(21)
inds = np.argsort(best_R2.mean(axis=0))
R2_sorted = best_R2[:, inds]
mean = R2_sorted.mean(axis=0)
mn = R2_sorted.min(axis=0, keepdims=True)
mx = R2_sorted.max(axis=0, keepdims=True)
plt.plot([-10, 30], [0, 0], '--', color='black')
plt.boxplot(
    R2_sorted, whis=[0, 100], widths=0.6,
    whiskerprops={'linewidth': 1.4},
    capprops={'linewidth': 1.4},
    boxprops={'linewidth': 1.4}
)
plt.xticks(ticks=t+1, labels=np.array(df_Y.columns)[inds], rotation=45, fontsize=16)
plt.yticks(np.linspace(-0.4, 1, 15), fontsize=16)
plt.ylim([-0.5, 1])
plt.xlim([0, 22])
plt.ylabel('$R^2$', fontsize=24)
plt.show()

# Preparing test data

In [ ]:
from psqi.predict import parse_geojson, compute_test_points, read_test_points, compute_predictions, save_predictions

In [ ]:
# Load geojson to reduce the grid to the region of interest
shp = parse_geojson('data/new_moscow.json', X_scaler=X_scaler)

# Compute grid of points within the region of interest with a given cell size in meters
test_X = compute_test_points(cell_size_m=10*1000, shp=shp, X_scaler=X_scaler)

In [ ]:
# To avoid computations again, read them from file instead
# test_X = read_test_points(10*1000)

# Making predictions

In [ ]:
u, s = compute_predictions(model=best_model, likelihood=best_likelihood, test_X=test_X)
df_pred = save_predictions(u=u, s=s, test_X=test_X, X_scaler=X_scaler, Y_scaler=Y_scaler, y_columns=df_Y.columns)

# Calculating PSQI

In [ ]:
from psqi.psqi import calc_psqi, save_psqi

psqi, conf, ws = calc_psqi(
    u=u, s=s, 
    norms=scaled_norms, 
    R2=best_R2,
    df_Y=df_Y,
)

In [ ]:
df_psqi = save_psqi(psqi=psqi, conf=conf, ws=ws, test_X=test_X, X_scaler=X_scaler)
df_psqi.head()